<a id='top'></a>
## Data Visualization: Discovering Trends and Patterns in an Online Retail Sales
### By: P. A. Ogbodum

<b> Contents Outline </b><br>
- <a href = '#introduction'> Introduction </a>
- <a href = '#data wrangling'> Data Wrangling </a>
- <a href = '#eda'> Exploratory Data Analysis </a>
- <a href = '#conclusion'> Conclusion </a>


<a id='introduction'></a>

# Introduction

**Data Set:** Online Retails Sale Dataset

**Source:** [kaggle](https://www.kaggle.com/datasets/rohitmahulkar/online-retails-sale-dataset)

**Description:** This data set was collected by an intern at Forage and contains information about online retail sales made by the Tata Group multinational conglomerate headquartered in Mumbai, India. It has 10 columns and 541,909 records for sales between 2010 and 2011.

**Variables:** There are 10 variables contained in this data set as listed below: </br>
- **InvoiceNo**: Unique ID to identify sale
- **InvoiceDate**: The date the invoice was made 
- **InvoiceTime**: The time the invoice was made
- **StockCode**: Code number for the item of purchase
- **Description**: Description about the type of the product
- **Quantity**: Quantity of the items purchased
- **UnitPrice**: Price of a single unit of the item
- **Totalsale**: Sum total of item purchased
- **CustomerID**: Unique number of identification for each customer
- **Country**: Country where the purchase was made

**Credit:** *ROHIT MAHULKAR*

**Analysis Question:** Best performing product in terms of sales between 2010-2011 and its characteristics.

**<a href='#top'>Go to first cell</a>**


In [1]:
# import needed modules
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import mitosheet
import re

%matplotlib inline

<a id='data wrangling'></a>
**<a href='#top'>Go to first cell</a>**
    
# Data Wrangling

> ## Data Gathering

In [2]:
# read flat file into jupyter
df = pd.read_csv('Online Retail.csv')

# check if operation was successful
df.sample(n=5)

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country
35607,539436,17-12-2010,02:49:00 PM,85014B,RED RETROSPOT UMBRELLA,1,12.72,12.72,NaN,United Kingdom
158091,550274,15-04-2011,12:23:00 PM,22668,PINK BABY BUNTING,5,2.95,14.75,15498.0,United Kingdom
361207,568331,26-09-2011,02:25:00 PM,22065,CHRISTMAS PUDDING TRINKET POT,48,0.39,18.72,16600.0,United Kingdom
311718,564315,24-08-2011,01:07:00 PM,22630,DOLLY GIRL LUNCH BOX,1,1.95,1.95,14730.0,United Kingdom
99862,544790,23-02-2011,12:27:00 PM,21983,PACK OF 12 BLUE PAISLEY TISSUES,12,0.29,3.48,16940.0,United Kingdom


> ## Data Assessment
>> ### Visual Assessment

In [3]:
# use mitosheet to perform visual assessment
mitosheet.sheet(analysis_to_replay="id-ywyefybwwh")

MitoWidget(analysis_data_json='{"analysisName": "id-qhwijlqgao", "analysisToReplay": {"analysisName": "id-ywye…

In [4]:
from mitosheet import *; register_analysis("id-ywyefybwwh");
    
# Imported Online Retail.csv
import pandas as pd
Online_Retail = pd.read_csv(r'Online Retail.csv')

# Reordered column Quantity
Online_Retail_columns = [col for col in Online_Retail.columns if col != 'Quantity']
Online_Retail_columns.insert(6, 'Quantity')
Online_Retail = Online_Retail[Online_Retail_columns]

# Reordered column Description
Online_Retail_columns = [col for col in Online_Retail.columns if col != 'Description']
Online_Retail_columns.insert(3, 'Description')
Online_Retail = Online_Retail[Online_Retail_columns]

# Sorted InvoiceDate in descending order
Online_Retail = Online_Retail.sort_values(by='InvoiceDate', ascending=False, na_position='last')

# Changed InvoiceDate to dtype datetime
import pandas as pd
Online_Retail['InvoiceDate'] = pd.to_datetime(Online_Retail['InvoiceDate'], infer_datetime_format=True, errors='coerce')


In [5]:
from mitosheet import *; register_analysis("id-azmjrmwwda");
    
# Imported Online Retail.csv
import pandas as pd
Online_Retail = pd.read_csv(r'Online Retail.csv')

# Pivoted into Online_Retail
Online_Retail_pivot = pd.DataFrame(data={})

# Deleted Online_Retail_pivot
del Online_Retail_pivot


<b>Notes :</b>
- Unideal data type for InvoiceDate and InvoiceTime columns [quality issue].
- Irrational values (-tve) values for Quantity and TotalSale columns [quality issue].
- Discrepancies in InvoiceNo values (some alphaNumeric), seem to denote Quantity and TotalSale rows with negative values [quality issue].

>> ### Programmatic Assessment

In [6]:
# dataframe overview
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   InvoiceDate  541909 non-null  object 
 2   InvoiceTime  541909 non-null  object 
 3   StockCode    541909 non-null  object 
 4   Description  540455 non-null  object 
 5   Quantity     541909 non-null  int64  
 6   UnitPrice    541909 non-null  float64
 7   Totalsale    541909 non-null  float64
 8   CustomerID   406829 non-null  float64
 9   Country      541909 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 41.3+ MB


In [7]:
df[df.Description.isna()]

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country
622,536414,01-12-2010,11:52:00 AM,22139,NaN,56,0.0,0.0,NaN,United Kingdom
1970,536545,01-12-2010,02:32:00 PM,21134,NaN,1,0.0,0.0,NaN,United Kingdom
1971,536546,01-12-2010,02:33:00 PM,22145,NaN,1,0.0,0.0,NaN,United Kingdom
1972,536547,01-12-2010,02:33:00 PM,37509,NaN,1,0.0,0.0,NaN,United Kingdom
1987,536549,01-12-2010,02:34:00 PM,85226A,NaN,1,0.0,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...,...,...
535322,581199,07-12-2011,06:26:00 PM,84581,NaN,-2,0.0,0.0,NaN,United Kingdom
535326,581203,07-12-2011,06:31:00 PM,23406,NaN,15,0.0,0.0,NaN,United Kingdom
535332,581209,07-12-2011,06:35:00 PM,21620,NaN,6,0.0,0.0,NaN,United Kingdom
536981,581234,08-12-2011,10:33:00 AM,72817,NaN,27,0.0,0.0,NaN,United Kingdom


<b>Notes:</b>
- Missing values in Description and CustomerID columns [quality issue].
- CustomerID should be integer data type not float64 [quality issue].

In [8]:
# check for duplicates
df.duplicated().any()

True

In [9]:
# get sum of all duplicates
df.duplicated().sum()

5268

In [10]:
# confirm number of duplicated rows 
df_dup = df[df.duplicated(keep='first')]
df_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5268 entries, 517 to 541701
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    5268 non-null   object 
 1   InvoiceDate  5268 non-null   object 
 2   InvoiceTime  5268 non-null   object 
 3   StockCode    5268 non-null   object 
 4   Description  5268 non-null   object 
 5   Quantity     5268 non-null   int64  
 6   UnitPrice    5268 non-null   float64
 7   Totalsale    5268 non-null   float64
 8   CustomerID   5225 non-null   float64
 9   Country      5268 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 452.7+ KB


<b>Notes:</b>
- There exist 5268 duplicated records in the dataframe [tidiness issue].

In [11]:
# Summary statistics
df.describe()

,Quantity,UnitPrice,Totalsale,CustomerID
count,541909.000000,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,17.987795,15287.690570
std,218.081158,96.759853,378.810824,1713.600303
min,-80995.000000,-11062.060000,-168469.600000,12346.000000
25%,1.000000,1.250000,3.400000,13953.000000
50%,3.000000,2.080000,9.750000,15152.000000
75%,10.000000,4.130000,17.400000,16791.000000
max,80995.000000,38970.000000,168469.600000,18287.000000


<b>Notes:</b>
- Quantity and Totalsale columns have their minimum value as the negative of their maximum value [quality issue].
- Although,the mean:max value ratio is significantly high suggesting the presence of outliers in the *Quantity, UnitPrice, and Totalsale* columns. This might be overlooked given the context of these values [quality issue].

In [12]:
# See and compare Quantity and Totalsale columns with minimum and maximum to see why they are inversely related  
check_df = df[(df.Quantity == 80995) | (df.Quantity == -80995)]
check_df

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country
540421,581483,09-12-2011,09:15:00 AM,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2.08,168469.6,16446.0,United Kingdom
540422,C581484,09-12-2011,09:27:00 AM,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2.08,-168469.6,16446.0,United Kingdom


In [13]:
# check to see attributes associated with InvoiceNo values have 'C'
c_values = df[df.InvoiceNo.str.contains('C')]

# check to see if all InvoiceNo 'C' values have negative Quantity entries
c_values.sample(n=10)

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country
76692,C542644,31-01-2011,11:33:00 AM,84678,CLASSICAL ROSE SMALL VASE,-2,1.25,-2.50,15497.0,United Kingdom
155744,C550013,14-04-2011,10:49:00 AM,22757,LARGE RED BABUSHKA NOTEBOOK,-1,1.25,-1.25,12982.0,United Kingdom
72415,C542245,26-01-2011,03:06:00 PM,22383,LUNCH BAG SUKI DESIGN,-1,1.65,-1.65,13098.0,United Kingdom
501804,C578832,25-11-2011,03:18:00 PM,21981,PACK OF 12 WOODLAND TISSUES,-7,0.39,-2.73,13069.0,United Kingdom
303758,C563553,17-08-2011,01:14:00 PM,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-1,2.95,-2.95,16755.0,United Kingdom
422516,C573092,27-10-2011,02:38:00 PM,23207,LUNCH BAG ALPHABET DESIGN,-66,1.45,-95.70,13791.0,United Kingdom
424227,C573244,28-10-2011,12:08:00 PM,22968,ROSE COTTAGE KEEPSAKE BOX,-2,9.95,-19.90,14623.0,United Kingdom
201889,C554338,23-05-2011,05:29:00 PM,20914,SET/5 RED RETROSPOT LID GLASS BOWLS,-2,2.95,-5.90,14038.0,United Kingdom
73417,C542346,27-01-2011,12:00:00 PM,22467,GUMBALL COAT RACK,-3,2.10,-6.30,12709.0,Germany
153833,C549827,12-04-2011,12:55:00 PM,POST,POSTAGE,-2,3.00,-6.00,12463.0,Belgium


In [14]:
# see number of rows in this sub-dataframe
len(c_values.InvoiceNo)

9288

<b>Rationale:</b> The number of rows from *c_values* (9288) compared with *df_dup* (5268) shows that not all the values in *c_values* are duplicated.

In [15]:
# sub-df to filter InvoiceNo where it contains values having "C" character and duplicated rows for the dataframe
c_and_dup = df[(df.InvoiceNo.str.contains('C')) & (df.duplicated())]

# get sample of dataframe above
c_and_dup.sample(n=8)

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country
384042,C570099,07-10-2011,12:11:00 PM,22064,PINK DOUGHNUT TRINKET POT,-1,1.45,-1.45,13798.0,United Kingdom
24177,C538341,10-12-2010,02:03:00 PM,22727,ALARM CLOCK BAKELIKE RED,-1,3.75,-3.75,15514.0,United Kingdom
361733,C568370,26-09-2011,04:43:00 PM,90148,LONG SILVER NECKLACE PASTEL FLOWER,-1,12.75,-12.75,15154.0,United Kingdom
133110,C547725,25-03-2011,10:43:00 AM,84929,ASSTD FRUIT+FLOWERS FRIDGE MAGNETS,-144,0.55,-79.20,NaN,United Kingdom
293157,C562582,07-08-2011,01:53:00 PM,21452,TOADSTOOL MONEY BOX,-1,2.95,-2.95,15640.0,United Kingdom
235951,C557663,21-06-2011,05:59:00 PM,21121,SET/10 RED POLKADOT PARTY CANDLES,-24,1.25,-30.00,NaN,EIRE
86890,C543611,10-02-2011,02:38:00 PM,82483,WOOD 2 DRAWER CABINET WHITE FINISH,-1,4.95,-4.95,17850.0,United Kingdom
407565,C571893,19-10-2011,02:13:00 PM,22720,SET OF 3 CAKE TINS PANTRY DESIGN,-3,4.95,-14.85,13314.0,United Kingdom


In [16]:
# get length of dataframe
len(c_and_dup)

37

<b>Rationale:</b> <i>since I can't get the meaning of <b>C</b> in the <b>InvoiceNo</b> column, I will mask it out for this analysis.</i>

**Summary of Assessment:**
<hr>

**Quality issues**
- Unideal data type for InvoiceDate and InvoiceTime columns.
- Irrational values (-tve) values for Quantity and TotalSale columns.
- Discrepancies in InvoiceNo values (some alphaNumeric), seem to denote Quantity and TotalSale rows with negative values.
- Missing values in Description and CustomerID columns.
- CustomerID should be integer data type not float64.
- Quantity and Totalsale columns have their minimum value as the negative of their maximum value.
- Although,the mean:max value ratio is significantly high suggesting the presence of outliers in the Quantity, UnitPrice, and Totalsale columns. This might be overlooked given the context of these values. (will be addressed during exploratory analysis).

**Tidiness issue**
- There exist 5268 duplicated records in the dataframe.

> ## Data Cleaning
*Next I will clean the data employing the Define-Code-Test Technique*

## Define
<br>
<hr>

<ol>
<li> Convert *InvoiceDate and InvoiceTime* columns data types from object (string) to datatime.
    <li> Drop negative values in <i> Quantity and Totalsale </i> columns.
<li> Drop rows having 'c' prefixed values in the *InvoiceNo* column.
<li> Enter the texts 'Not Available' for missing values in the Description column and the number zero (0) for missing values in the *CustomerID* column.
<li> Change *CustomerID* data type from float to integer.
<li> Drop duplicated rows.
</ol>

## Code
<br>
<hr>

In [17]:
# duplicate dataframe to easily access original copy in the event of error.
df_clean = df.copy()

# test
# df_clean.sample(n=10)

<b> 1. Convert *InvoiceDate and InvoiceTime* columns data types from object (string) to datatime. </b>

In [18]:
# convert *InvoiceDate & InvoiceTime* to datetime data type
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'], infer_datetime_format=True)
df_clean['InvoiceTime'] = pd.to_datetime(df_clean['InvoiceTime'], infer_datetime_format=True)

# df_clean['InvoiceTime'] = pd.to_datetime(df_clean['InvoiceTime'], 
#                                                        infer_datetime_format=True, errors='coerce')

C:\Users\PRINCE\miniconda3\envs\retail_env\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13-12-2010' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\PRINCE\miniconda3\envs\retail_env\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '14-12-2010' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\PRINCE\miniconda3\envs\retail_env\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15-12-2010' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\PRINCE\miniconda3\envs\retail_env\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarn

<b> 2. Drop negative values in *Quantity and TotalSale* columns. </b>

In [19]:
# filter all rows where *Quantity & Totalsale* columns have negative values
neg_qty = df_clean[(df_clean.Quantity < 0) | 
                   (df_clean.Totalsale < 0)].index

# drop negative rows
df_clean.drop(neg_qty, inplace=True)

<b> 3. Drop rows having 'c' prefixed values in the *InvoiceNo* column </b>

*There exist no values in the InvoiceNo column with 'C' prefixed after droping all rows in the Quantity column having negative values*

<b> 4. Enter the texts 'NOT AVAILABLE' for missing values in the Description column and the number zero (0) for missing values in the *CustomerID* column. </b>

In [20]:
# fill NA value with NOT AVAILABLE
df_clean['Description'] = df_clean['Description'].fillna('NOT AVAILABLE')

# fill NA value with zero (0)
df_clean['CustomerID'] = df_clean['CustomerID'].fillna(0)

<b> 5. Change *CustomerID* data type from float to integer </b>

In [21]:
# convert from float to integer data type
df_clean.CustomerID = df_clean.CustomerID.astype('int64')

<b> 6. Drop duplicated rows. </b>

In [22]:
# drop duplicates
df_clean.drop_duplicates(keep=False, inplace=True)

## Test
<br>
<hr>


<b> 1. Confirm that *InvoiceDate and InvoiceTime* columns data types have been converted from object (string) to datatime. </b>

In [23]:
df_clean.dtypes

InvoiceNo              object
InvoiceDate    datetime64[ns]
InvoiceTime    datetime64[ns]
StockCode              object
Description            object
Quantity                int64
UnitPrice             float64
Totalsale             float64
CustomerID              int64
Country                object
dtype: object

<b> 2. Confirm that negative values in *Quantity and TotalSale* columns have been dropped. </b>

In [24]:
test_df = df_clean[ (df_clean.Quantity < 0) |
                  (df_clean.Totalsale < 0) ]

test_df.head()

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country


<b> 3. Confirm that there exist no 'c' prefixed values in the *InvoiceNo* column. </b>

In [25]:
# create lists to store respective values
no_c = []
contain_c = []
for i in df_clean.InvoiceNo:
    if i in 'C':
        contain_c.append(i)
    else:
        no_c.append(i)
        
# print list for values containing 'C'
contain_c

# the list is empty, confirming that there exist no 'C' value in the InvoiceNo column.

[]

<b> 4. Confirm that missing values in Description and CustomerID column have been filled.</b>

In [26]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 521205 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    521205 non-null  object        
 1   InvoiceDate  521205 non-null  datetime64[ns]
 2   InvoiceTime  521205 non-null  datetime64[ns]
 3   StockCode    521205 non-null  object        
 4   Description  521205 non-null  object        
 5   Quantity     521205 non-null  int64         
 6   UnitPrice    521205 non-null  float64       
 7   Totalsale    521205 non-null  float64       
 8   CustomerID   521205 non-null  int64         
 9   Country      521205 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(4)
memory usage: 43.7+ MB


<b> 5. check that column data type have been changed from float to integer </b>

In [27]:
# check that column data type have been changed from float to integer
df_clean.CustomerID.dtype

dtype('int64')

<b> 6. check that there's no duplicate row. </b>

In [28]:
# check if there's any duplicate
df_clean.duplicated().any()

False

> ### Data Storing

In [29]:
# export cleaned dataframe
df_clean.to_csv('df_cleaned.csv')

<a id='eda'></a>
**<a href='#top'>Go to first cell</a>**
    
# Exploratory Data Analysis

Address issue of outliers.

In [30]:
# create a copy of cleaned dataframe to use for EDA
dfc = df_clean.copy()

In [31]:
dfc.sample(7)

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country
245672,558625,2011-06-30,2022-10-11 17:29:00,22273,FELTCRAFT DOLL MOLLY,6,2.95,17.70,14132,United Kingdom
497702,578463,2011-11-24,2022-10-11 12:30:00,23367,SET 12 COLOUR PENCILS SPACEBOY,16,0.65,10.40,12757,Portugal
129779,547389,2011-03-22,2022-10-11 16:08:00,22960,JAM MAKING SET WITH JARS,2,4.25,8.50,16721,United Kingdom
158244,550279,2011-04-15,2022-10-11 13:16:00,22352,LUNCH BOX WITH CUTLERY RETROSPOT,12,2.55,30.60,14700,United Kingdom
99942,544795,2011-02-23,2022-10-11 13:21:00,22037,ROBOT BIRTHDAY CARD,12,0.42,5.04,15858,United Kingdom
380748,569834,2011-06-10,2022-10-11 13:00:00,85071A,BLUE CHARLIE+LOLA PERSONAL DOORSIGN,24,0.39,9.36,14383,United Kingdom
190813,553210,2011-05-16,2022-10-11 09:40:00,21915,RED HARMONICA IN BOX,24,1.25,30.00,14016,EIRE


<a id='conclusion'></a>
**<a href='#top'>Go to first cell</a>**
    
# Conclusion